# Step 3: Import the orders_products_combined DataFrame from the pickle file

In [1]:
# Import the necessary libraries
import pandas as pd
import os

# Define the file path
prepared_data_path = '/Users/dela/Documents/15-01-2025 Instacart Basket Analysis/02 Data/Prepared Data/'
df_orders_products_combined = pd.read_pickle(os.path.join(prepared_data_path, 'orders_products_combined.pkl'))

# Check the first few rows
print(df_orders_products_combined.head())


   order_id  user_id  order_number  order_day_of_week  order_hour_of_day  \
0   2539329        1             1                  2                  8   
1   2539329        1             1                  2                  8   
2   2539329        1             1                  2                  8   
3   2539329        1             1                  2                  8   
4   2539329        1             1                  2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered _merge  
0                     NaN         196                  1          0   both  
1                     NaN       14084                  2          0   both  
2                     NaN       12427                  3          0   both  
3                     NaN       26088                  4          0   both  
4                     NaN       26405                  5          0   both  


# Step 4: Check the shape of the imported DataFrame

In [3]:
print(df_orders_products_combined.shape)

#dimensions are matched the original merged DataFrame.

(32434489, 10)


# Step 5: Combine the orders_products_combined DataFrame with products data set

In [7]:
#import the cleaned products DataFrame (df_prods)
df_prods = pd.read_csv(os.path.join(prepared_data_path, 'products_checked.csv'))
print(df_prods.head())


   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  prices  
0             19     5.8  
1             13     9.3  
2              7     4.5  
3              1    10.5  
4             13     4.3  


In [13]:
# Check Column Names in Both DataFrames
print(df_orders_products_combined.columns)
print(df_prods.columns)


Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', '_merge'],
      dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')


In [15]:
#Data Type Mismatch

df_orders_products_combined['product_id'] = df_orders_products_combined['product_id'].astype(str)
df_prods['product_id'] = df_prods['product_id'].astype(str)

In [17]:
#Missing or NaN Values check
print(df_orders_products_combined['product_id'].isnull().sum())
print(df_prods['product_id'].isnull().sum())


0
0


In [21]:
df_orders_products_combined['product_id'] = df_orders_products_combined['product_id'].astype(str)
df_prods['product_id'] = df_prods['product_id'].astype(str)


In [23]:
print("Missing values in df_orders_products_combined:", df_orders_products_combined['product_id'].isnull().sum())
print("Missing values in df_prods:", df_prods['product_id'].isnull().sum())


Missing values in df_orders_products_combined: 0
Missing values in df_prods: 0


In [25]:
#column overlap check
common_ids = set(df_orders_products_combined['product_id']).intersection(set(df_prods['product_id']))
print(f"Number of common product IDs: {len(common_ids)}")


Number of common product IDs: 49659


In [27]:
df_orders_products_combined['product_id'] = df_orders_products_combined['product_id'].astype(str)
df_prods['product_id'] = df_prods['product_id'].astype(str)


In [29]:
df_final_merge = df_orders_products_combined.merge(
    df_prods, on='product_id', how='inner', indicator=True
)


ValueError: Cannot use name of an existing column for indicator column

# Steps to Resolve the issue with merging

In [31]:
# Check and Align Data Types
df_orders_products_combined['product_id'] = df_orders_products_combined['product_id'].astype(str)
df_prods['product_id'] = df_prods['product_id'].astype(str)


In [33]:
# Check for Overlapping Columns
print("Columns in df_orders_products_combined:", df_orders_products_combined.columns)
print("Columns in df_prods:", df_prods.columns)


Columns in df_orders_products_combined: Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', '_merge'],
      dtype='object')
Columns in df_prods: Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')


In [35]:
print(df_orders_products_combined['product_id'].dtype)
print(df_prods['product_id'].dtype)


object
object


In [37]:
#Check for Whitespace or Formatting Issues
df_orders_products_combined['product_id'] = df_orders_products_combined['product_id'].str.strip()
df_prods['product_id'] = df_prods['product_id'].str.strip()


In [39]:
#Verify Common Product IDs
common_ids = set(df_orders_products_combined['product_id']).intersection(set(df_prods['product_id']))
print(f"Number of common product IDs: {len(common_ids)}")
print(f"Total unique product IDs in df_orders_products_combined: {df_orders_products_combined['product_id'].nunique()}")
print(f"Total unique product IDs in df_prods: {df_prods['product_id'].nunique()}")


Number of common product IDs: 49659
Total unique product IDs in df_orders_products_combined: 49677
Total unique product IDs in df_prods: 49670


In [41]:
# Check for Duplicates in product_id Columns
# Check for duplicates in df_orders_products_combined
print(f"Duplicates in df_orders_products_combined['product_id']: {df_orders_products_combined['product_id'].duplicated().sum()}")

# Check for duplicates in df_prods
print(f"Duplicates in df_prods['product_id']: {df_prods['product_id'].duplicated().sum()}")


Duplicates in df_orders_products_combined['product_id']: 32384812
Duplicates in df_prods['product_id']: 2


In [43]:
# Remove duplicates while keeping the first occurrence
df_orders_products_combined = df_orders_products_combined.drop_duplicates(subset=['product_id', 'order_id'])
print(f"New shape of df_orders_products_combined: {df_orders_products_combined.shape}")


New shape of df_orders_products_combined: (32434489, 10)


In [45]:
# Drop duplicates in df_prods
df_prods = df_prods.drop_duplicates(subset=['product_id'])
print(f"New shape of df_prods: {df_prods.shape}")


New shape of df_prods: (49670, 5)


# Recheck for Null Values

In [47]:
print(f"Null values in df_orders_products_combined['product_id']: {df_orders_products_combined['product_id'].isnull().sum()}")
print(f"Null values in df_prods['product_id']: {df_prods['product_id'].isnull().sum()}")


Null values in df_orders_products_combined['product_id']: 0
Null values in df_prods['product_id']: 0


In [51]:
# columns are consistent in type and formatting
# Ensure column consistency in type and format using .loc to avoid SettingWithCopyWarning
df_orders_products_combined['product_id'] = df_orders_products_combined['product_id'].astype(str).str.strip()
df_prods.loc[:, 'product_id'] = df_prods['product_id'].astype(str).str.strip()


In [57]:
print(df_orders_products_combined['product_id'].dtype)
print(df_prods['product_id'].dtype)


object
object


In [59]:
print(df_orders_products_combined['product_id'].isnull().sum())
print(df_prods['product_id'].isnull().sum())


0
0


In [61]:
common_ids = set(df_orders_products_combined['product_id']).intersection(set(df_prods['product_id']))
print(f"Number of common product IDs: {len(common_ids)}")
print(f"Unique product IDs in df_orders_products_combined: {df_orders_products_combined['product_id'].nunique()}")
print(f"Unique product IDs in df_prods: {df_prods['product_id'].nunique()}")


Number of common product IDs: 49659
Unique product IDs in df_orders_products_combined: 49677
Unique product IDs in df_prods: 49670


# Investigate Missing IDs

In [63]:
# IDs in df_orders_products_combined but not in df_prods
missing_in_prods = set(df_orders_products_combined['product_id']) - set(df_prods['product_id'])
print(f"Product IDs in df_orders_products_combined but not in df_prods: {len(missing_in_prods)}")
print(missing_in_prods)

# IDs in df_prods but not in df_orders_products_combined
missing_in_orders = set(df_prods['product_id']) - set(df_orders_products_combined['product_id'])
print(f"Product IDs in df_prods but not in df_orders_products_combined: {len(missing_in_orders)}")
print(missing_in_orders)


Product IDs in df_orders_products_combined but not in df_prods: 18
{'69', '262', '3736', '38183', '525', '4283', '40440', '116', '2240', '3230', '6799', '26519', '4790', '34', '2586', '1780', '1511', '3159'}
Product IDs in df_prods but not in df_orders_products_combined: 11
{'3630', '3718', '25383', '43725', '27499', '49540', '46625', '36233', '45971', '7045', '37703'}


# Decided for Exclude Missing IDs

In [67]:
# Filter out rows with missing IDs
df_orders_products_combined = df_orders_products_combined[
    ~df_orders_products_combined['product_id'].isin(missing_in_prods)
]
df_prods = df_prods[~df_prods['product_id'].isin(missing_in_orders)]


In [69]:
df_final_merge = df_orders_products_combined.merge(
    df_prods, on='product_id', how='inner', indicator=True
)

# Preview the merged DataFrame
print(df_final_merge.head())

# Check the merge flag values
print(df_final_merge['_merge'].value_counts())


ValueError: Cannot use name of an existing column for indicator column

In [71]:
# Check for the '_merge' column in both DataFrames
print("_merge column in df_orders_products_combined:", '_merge' in df_orders_products_combined.columns)
print("_merge column in df_prods:", '_merge' in df_prods.columns)


_merge column in df_orders_products_combined: True
_merge column in df_prods: False


In [74]:
# Drop the _merge column in df_orders_products_combined
df_orders_products_combined = df_orders_products_combined.drop(columns=['_merge'])


In [76]:
#check
print("_merge column in df_orders_products_combined after dropping:", '_merge' in df_orders_products_combined.columns)


_merge column in df_orders_products_combined after dropping: False


In [78]:
# Perform the merge again
df_final_merge = df_orders_products_combined.merge(
    df_prods, on='product_id', how='inner', indicator=True
)

# Preview the merged DataFrame
print(df_final_merge.head())

# Check the merge flag values
print(df_final_merge['_merge'].value_counts())


   order_id  user_id  order_number  order_day_of_week  order_hour_of_day  \
0   2539329        1             1                  2                  8   
1   2539329        1             1                  2                  8   
2   2539329        1             1                  2                  8   
3   2539329        1             1                  2                  8   
4   2539329        1             1                  2                  8   

   days_since_prior_order product_id  add_to_cart_order  reordered  \
0                     NaN        196                  1          0   
1                     NaN      14084                  2          0   
2                     NaN      12427                  3          0   
3                     NaN      26088                  4          0   
4                     NaN      26405                  5          0   

                              product_name  aisle_id  department_id  prices  \
0                                     Soda 

# Step 6: Confirm the Results of the Merge Using the Merge Flag

In [80]:
print(df_final_merge['_merge'].value_counts())


_merge
both          32404289
left_only            0
right_only           0
Name: count, dtype: int64


In [82]:
# Drop the _merge Column 
df_final_merge.drop(columns=['_merge'], inplace=True)


# Step 7: Export the Newly Created DataFrame as ords_prods_merge

In [84]:
# Export as Pickle Format
df_final_merge.to_pickle(os.path.join(prepared_data_path, 'ords_prods_merge.pkl'))


In [86]:
# Confirm Pickle Export
df_check_pickle = pd.read_pickle(os.path.join(prepared_data_path, 'ords_prods_merge.pkl'))
print(df_check_pickle.head())



   order_id  user_id  order_number  order_day_of_week  order_hour_of_day  \
0   2539329        1             1                  2                  8   
1   2539329        1             1                  2                  8   
2   2539329        1             1                  2                  8   
3   2539329        1             1                  2                  8   
4   2539329        1             1                  2                  8   

   days_since_prior_order product_id  add_to_cart_order  reordered  \
0                     NaN        196                  1          0   
1                     NaN      14084                  2          0   
2                     NaN      12427                  3          0   
3                     NaN      26088                  4          0   
4                     NaN      26405                  5          0   

                              product_name  aisle_id  department_id  prices  
0                                     Soda  